In [2]:
import duckdb

from pyproj import CRS

from lonboard import Map, PolygonLayer
from lonboard.basemap import CartoBasemap

In [3]:
db = duckdb.connect(database='caba_population_analysis.duckdb', read_only=False)
db.execute("INSTALL spatial; LOAD spatial;")
db.execute("INSTALL httpfs; LOAD httpfs;")

In [4]:
crs_4326 = CRS.from_epsg(4326)

#### Census 2010 data

In [5]:
census_radius_2010 = r"https://cdn.buenosaires.gob.ar/datosabiertos/datasets/direccion-general-de-estadisticas-y-censos/informacion-censal-por-radio/caba_radios_censales.geojson"

In [6]:
db.sql(f"""
CREATE OR REPLACE TABLE census_blocks_2010 AS
    SELECT *
    FROM ST_Read('{census_radius_2010}');
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [7]:
db.sql("""
    describe census_blocks_2010;
""")

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ WKT         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ ID          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ CO_FRAC_RA  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ COMUNA      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ FRACCION    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ RADIO       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ TOTAL_POB   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ T_VARON     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ T_MUJER     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ T_VIVIENDA  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ V_PARTICUL  │ VARC

#### Census 2001 data

In [8]:
# db.sql("""
# -- Clip source_polygons using clip_polygons
# CREATE TABLE clipped_result AS
# SELECT
#     s.id AS source_id,
#     c.id AS clip_id,
#     ST_Intersection(s.geom, c.geom) AS geom
# FROM
#     source_polygons s
# JOIN
#     clip_polygons c
# ON
#     ST_Intersects(s.geom, c.geom)
# WHERE
#     ST_IsValid(s.geom) AND ST_IsValid(c.geom);
#            """)

https://www.flother.is/til/duckdb-st-read/

In [9]:
census_blocks_2001_shp = r"https://cdn.buenosaires.gob.ar/datosabiertos/datasets/direccion-general-de-estadisticas-y-censos/informacion-censal-por-radio/informacion-censal-por-radio-2001.zip"

In [10]:
db.sql("""
CREATE OR REPLACE TABLE census_blocks_2001 AS
    SELECT *
    --from st_read('/vsicurl/census_blocks_2001_shp/informacion_censal_por_radio_2001.shp', open_options=['GDAL_HTTP_UNSAFESSL=YES'])
    FROM ST_Read('/vsizip/informacion-censal-por-radio-2001.zip/informacion_censal_por_radio_2001.shp')
""")

In [11]:
# todo: read content directly from zip file without unzipping
blocks_2001_crs = 'PROJCS["Argentina_GKBsAs",GEOGCS["GCS_Campo_Inchauspe",DATUM["D_Campo_Inchauspe",SPHEROID["International_1924",6378388.0,297.0]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",100000.0],PARAMETER["False_Northing",100000.0],PARAMETER["Central_Meridian",-58.4627],PARAMETER["Scale_Factor",0.999998],PARAMETER["Latitude_Of_Origin",-34.6297166],UNIT["Meter",1.0]]'

In [12]:
db.sql("""
    summarize census_blocks_2001;
""")

┌─────────────┬─────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬───────┬─────────────────┐
│ column_name │ column_type │                                                                                                                                                                                                    min      

In [13]:
blocks_2010 = PolygonLayer.from_duckdb(
    sql="SELECT geom FROM census_blocks_2010",
    con=db,
    get_fill_color=[50, 25, 0, 50],
    get_line_color=[255, 0, 0],
    get_line_width=1,
    pickable=True,
    extruded=False,
    auto_highlight=True
)

blocks_2001 = PolygonLayer.from_duckdb(
    sql=f"SELECT st_transform(geom,'{blocks_2001_crs}', 'EPSG:4326', always_xy:=TRUE) as geom FROM census_blocks_2001",
    con=db,
    get_fill_color=[200, 0, 100, 0],
    get_line_color=[0, 255, 0],
    get_line_width=1,
    pickable=True,
    extruded=False,
    auto_highlight=True
)

Map(
    layers=[blocks_2010, blocks_2001],
    basemap_style=CartoBasemap.Voyager
)

c:\Users\learo\MyDocs\GitHub\geopython\.venv\Lib\site-packages\lonboard\_geoarrow\ops\reproject.py:37: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.Voyager: 'https://basemaps.cartocdn.com/gl/voyager-gl-style/style.json'>, cust…

Aggregate the blocks in Fracciones

In [17]:
fraccion_2010_sql = """
SELECT
    FRACCION,
    ST_Union_Agg(geom) AS geom,
    SUM(TOTAL_POB::INT) AS total_population
FROM
    census_blocks_2010
GROUP BY
    FRACCION
"""

In [18]:
fraccion_2001_sql = f"""
SELECT
    FRAC,
    ST_Transform(ST_Union_Agg(geom), '{blocks_2001_crs}', 'EPSG:4326', always_xy:=TRUE) AS geom,
    SUM(POB_TOT) AS total_population
FROM
    census_blocks_2001
GROUP BY
    FRAC
"""

In [19]:
db.sql(f"""CREATE OR REPLACE VIEW v_fraccion_2010 AS {fraccion_2010_sql}""")
db.sql(f"""CREATE OR REPLACE VIEW v_fraccion_2001 AS {fraccion_2001_sql}""")

In [20]:
fracciones_2010 = PolygonLayer.from_duckdb(
    sql=fraccion_2010_sql,
    con=db,
    get_fill_color=[50, 25, 0, 50],
    get_line_color=[255, 0, 0],
    get_line_width=1,
    pickable=True,
    extruded=False,
    auto_highlight=True
)

fracciones_2001 = PolygonLayer.from_duckdb(
    sql=fraccion_2001_sql,
    con=db,
    get_fill_color=[200, 0, 100, 0],
    get_line_color=[0, 255, 0],
    get_line_width=1,
    pickable=True,
    extruded=False,
    auto_highlight=True
)

Map(
    layers=[fracciones_2010, fracciones_2001],
    basemap_style=CartoBasemap.Voyager
)

Map(basemap_style=<CartoBasemap.Voyager: 'https://basemaps.cartocdn.com/gl/voyager-gl-style/style.json'>, cust…

In [ ]:
db.sql("""
CREATE OR REPLACE TABLE fraccion_population_change AS
SELECT
    fr2010.*,
    fr2001.total_population AS total_population_2001,
    ROUND(((fr2010.total_population - fr2001.total_population) / fr2001.total_population) * 100, 2) AS population_change
FROM
    v_fraccion_2010 AS fr2010
JOIN
    v_fraccion_2001 AS fr2001
ON
    ST_Intersects(ST_Centroid(fr2010.geom), fr2001.geom)
""")

In [28]:
db.table("fraccion_population_change").sort('population_change').limit(10)

┌──────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [29]:
from matplotlib import colormaps
from lonboard.colormap import apply_continuous_cmap

In [ ]:
cmap = apply_continuous_cmap(values=db.table("fraccion_population_change").arrow()["population_change"].to_numpy(),
                            cmap=colormaps['Blues'])

In [42]:
pop_change_lyr = PolygonLayer.from_duckdb(
    sql=db.table("fraccion_population_change"),
    con=db,
    get_fill_color=cmap,
    get_line_color=[0, 255, 0],
    get_line_width=1,
    pickable=True,
    extruded=False,
    auto_highlight=True
)

Map(
    layers=[pop_change_lyr],
    basemap_style=CartoBasemap.Voyager
)

c:\Users\learo\MyDocs\GitHub\geopython\.venv\Lib\site-packages\lonboard\_geoarrow\ops\reproject.py:37: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.Voyager: 'https://basemaps.cartocdn.com/gl/voyager-gl-style/style.json'>, cust…